<a href="https://colab.research.google.com/github/TerenceLiu98/Algorithm/blob/master/cryptanalysis_of_RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RSA Cryptosystem 

使得 $N = pq$ 是两个大质数 $p$ 和 $q$的乘积，使得 $P = C = Z_N$（以 $N$ 为模），定义密钥空间为:
<br><br>
$$K = \{(N, p, q, e, d): ed \equiv 1 \ (mod \ \phi(N))\}$$
<br>
其中，$\phi(N) = (p - 1)(q - 1)$ （*参见欧拉$\phi$函数* ）

* 加密：$enc_K(x) = x^e \ mod \ N$
* 解密：$dec_K(y) = y^d \ mod \ N$

* RSA 方程： $gcd(e, \phi(N)) = 1 \Rightarrow ed \equiv 1 \ (mod \ \phi(N)) \Rightarrow ed = 1 + k \phi(N)$
* RSA 模数：$N = p q$ 


## 如何生成大素数

### 素性测试

如果数字小，我们可以用枚举法，也就是俗称的「暴力破解」可解，可以写一个循环从$2$开始一直除到$n - 1$（实际上 $\sqrt{n}$即可），如果其中有数可以整除 $n$， 则 $n$ 不是素数。

第一种：$n$

```python 
import datetime 
start = datetime.datetime.now()
a = 0

for n in range(2, 50000):
    for x in range(2, n):
        if n % x == 0:
            break
    else:
        # loop fell through without finding a factor
        # print(n)
        a = a + 1

end = datetime.datetime.now() 
time = end - start
print(time, a)
```
 


In [0]:
import datetime 
start = datetime.datetime.now()
a = 0

for n in range(2, 50000):
    for x in range(2, n):
        if n % x == 0:
            break
    else:
        # loop fell through without finding a factor
        # print(n)
        a = a + 1

end = datetime.datetime.now() 
time = end - start
print(time, a)

0:00:13.897045 5133


### 费马小定理

> 费马小定理是数论中的一个定理：假设 $a$ 是一个整数，$p$ 是一个质数，那么 $a^p - a$ 是 $p$ 的倍数，可以表示为：$a^p \equiv a \ (mod \ p)$(同余：余数相同)；如果 $a$ 不是 $p$ 的倍数，这个定理也可以写成：$a^{p - 1} \equiv 1 \ (mod \ p)$
>
> 来源：维基百科

* 如果 $x$ 和 $y$ 的最大公约数为 $1$，则称：$x$ 和 $y$ 互素 (coprime)

* 根据费马小定理：$a(a^{p - 1} - 1)$ 是 $p$ 的倍数，所以，当 $a$ 不是 $p$ 的倍数时， $a^{p - 1} - 1$ 是 $p$ 的倍数 $\Rightarrow a^{p - 1} \equiv 1 \ (mod \ p)$ 
* 费马小定理为判定一个数是否为素数的**必要条件**，并非充分条件，就是说存在**伪素数**满足费马小定理却不是素数，例如：$2^340 \equiv 1 \ (mod \ 341)$，但是 $341 = 11 \times 31$

### 欧拉出现 -- 费马欧拉定理

* $a$ 和 $n$ 互素：$a \bot n \Rightarrow a^{\phi n} \equiv 1 \ (mod \ n) \textbf{ 费马欧拉定理 }; \phi(an) - \phi(m)\phi(n)$
* 当 $n$为质数时， $\phi n = (n - 1)$ 所以，$a^{\phi n} \equiv 1 \ (mod \ n) \Rightarrow a^{n - 1} \equiv 1 \ (mod \ n)$ 也就是费马小定理



### 费马素性检验

根据费马小定理，如果我们检验的 $a$ 越多，得到素数的概率会越大，但是得到合数的概率却不可能等于 $0$。 所以，我们称这些数是伪素数(pseudo-prime)

```python
ddef fermat_prime_test(n, k):
  if n == 2:
    print(n, "n is a composite")
  
  if n % 2 == 0:
    print(n, "n is a composite")
  
  for i in range(k):
    a = random.randint(1, n-1)
    if pow(a, n - 1) % n != 1:
      print(n, "n is a composite")
      break
      
  return True
      

In [0]:
def fermat_prime_test(n, k):
  import random
  
  if n == 2:
    print(n, "n is a composite")
  
  if n % 2 == 0:
    print(n, "n is a composite")
  
  for i in range(k):
    a = random.randint(1, n-1)
    if pow(a, n - 1) % n != 1:
      print(n, "n is a composite")
      break
      
  return 0

fermat_prime_test(6789865, 6789865)

6789865 n is a composite


0

### Miller-Rabin 素性测试 (probably prime)

基于费马小定理的二次探测定理：

1. Find $n - 1 = 2^k \times m$
2. choose $a: 1 < a < n - 1$
3. compute $b_0 = a^m \ (mod \ n), b_i = b_{i - 1}^2$


例子： Is $53$ prime? 

1. $n = 53 \Rightarrow n - 1 = 52; \frac{52}{2^1} = 26; \frac{52}{2^2} = 13; \frac{52}{2^3} = 6.5 \Rightarrow $ We choose $52 = 2^2 \times 13 \Rightarrow k = 2, m = 13$

2. $a$ is from $1$ to $52$, we can just choose $a = 2$ 
3. $b_0 = 2^{13} \ mod \ 53 \Rightarrow b_0 = 30 \neq (1 \ mod \ 53) \neq (- 1 \ mod \ 53)$
4. $b_1 = 30^2 \ mod \ 53 = -1 \ mod \ 53 = 52 \Rightarrow \textbf{probably prime}$

```
Input: n > 2, an odd integer to be tested for primality;
       k, a parameter that determines the accuracy of the test
Output: composite if n is composite, otherwise probably prime
write n − 1 as 2s·d with d odd by factoring powers of 2 from n − 1
LOOP: repeat k times:
   pick a randomly in the range [2, n − 1]
   x ← ad mod n
   if x = 1 or x = n − 1 then do next LOOP
   for r = 1 .. s − 1
      x ← x2 mod n
      if x = 1 then return composite
      if x = n − 1 then do next LOOP
   return composite
return probably prime
```


In [0]:
def miller_rabin(n, k):
    
    import random 

    if n == 2:
        return True

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True
  
miller_rabin(56787641, 7890)

False